# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 64

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:

from keras.preprocessing import image
from PIL import Image
  

In [6]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    img_idx = [i for i in range(0, 20)]
    
    x = 20 # number of images you use for each video
    y = 120 # image height
    z = 120 # image width
    
    
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        
        for batch in range(num_batches): # we iterate over the number of batches
            
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                
                for idx,item  in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    resized = imresize(image, (y,z))
                    
                    
                      
                    batch_data[folder,idx,:,:,0] = resized[:, : , 0]/255
                    batch_data[folder,idx,:,:,1] = resized[:, : , 1]/255
                    batch_data[folder,idx,:,:,2] = resized[:, : , 2]/255
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining = len(t) - num_batches*batch_size
        batch_data = np.zeros((remaining,x,y,z,3))
        batch_labels = np.zeros((remaining,5))
        for folder in range(remaining):
            imgs = os.listdir(source_path+"/"+t[num_batches*batch_size + folder].split(';')[0])
            for idx,item in enumerate(img_idx):
                
                image = imread(source_path+"/"+t[num_batches*batch_size + folder].split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                
                    
                resized = imresize(image, (y,z))
                    
                batch_data[folder,idx,:,:,0] = resized[:, : , 0]/255
                batch_data[folder,idx,:,:,1] = resized[:, : , 1]/255
                batch_data[folder,idx,:,:,2] = resized[:, : , 2]/255

            batch_labels[folder, int(t[folder + num_batches*batch_size].strip().split(';')[2])] = 1
        yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM
from keras.layers.convolutional import Conv3D,Conv2D, MaxPooling3D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers.recurrent import LSTM, GRU
from keras.optimizers import Adam

#### Lets start with simple model

In [9]:
model = Sequential()

model.add(Conv3D(16, (3, 3, 3),padding='same' ,input_shape=(20,120,120,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))



model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())

model.add(Dense(5,activation='softmax'))

In [10]:
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 20, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 60, 60, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 10, 60, 60, 32)    13856     
_________________________________________________________________
activation_2 (Activation)    (None, 10, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 10, 60, 60, 32)    128       
__________

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init1' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [14]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 64
Source path =  ./Project_data/train ; batch size = 64
Epoch 1/10
11/11 [==============================] - 63s 6s/step - loss: 1.2749 - categorical_accuracy: 0.5260 - val_loss: 3.4163 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init1_2020-12-2811_48_09.796980/model-00001-1.27542-0.51885-3.41630-0.35000.h5
Epoch 2/10
11/11 [==============================] - 46s 4s/step - loss: 0.5739 - categorical_accuracy: 0.8031 - val_loss: 3.2330 - val_categorical_accuracy: 0.4400

Epoch 00002: saving model to model_init1_2020-12-2811_48_09.796980/model-00002-0.58128-0.79940-3.23303-0.44000.h5
Epoch 3/10
11/11 [==============================] - 47s 4s/step - loss: 0.3479 - categorical_accuracy: 0.8854 - val_loss: 0.9639 - val_categorical_accuracy: 0.6500

Epoch 00003: saving model to model_init1_2020-12-2811_48_09.796980/model-00003-0.34972-0.88386-0.96395-0.65000.h5
Epoch 4/10
11/11 [==============================] - 46s 4

#### Model is clearly overfiting. Lets add some dropout into it.

In [15]:
model = Sequential()

model.add(Conv3D(16, (3, 3, 3),padding='same' ,input_shape=(20,120,120,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(32, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(128, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))



model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(5,activation='softmax'))

In [16]:
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 20, 120, 120, 16)  1312      
_________________________________________________________________
activation_5 (Activation)    (None, 20, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 20, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 60, 60, 16)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 60, 60, 16)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 10, 60, 60, 32)    13856     
_________________________________________________________________
activation_6 (Activation)    (None, 10, 60, 60, 32)    0         
__________

In [17]:
model_name = 'model_init2' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [18]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
11/11 [==============================] - 60s 5s/step - loss: 1.6797 - categorical_accuracy: 0.3993 - val_loss: 2.2353 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init2_2020-12-2811_48_09.796980/model-00001-1.69133-0.39970-2.23531-0.28000.h5
Epoch 2/10
11/11 [==============================] - 46s 4s/step - loss: 1.0710 - categorical_accuracy: 0.5588 - val_loss: 2.3183 - val_categorical_accuracy: 0.3800

Epoch 00002: saving model to model_init2_2020-12-2811_48_09.796980/model-00002-1.07765-0.55354-2.31828-0.38000.h5
Epoch 3/10
11/11 [==============================] - 47s 4s/step - loss: 0.9095 - categorical_accuracy: 0.6313 - val_loss: 1.8076 - val_categorical_accuracy: 0.4900

Epoch 00003: saving model to model_init2_2020-12-2811_48_09.796980/model-00003-0.90791-0.63499-1.80760-0.49000.h5
Epoch 4/10
11/11 [==============================] - 47s 4s/step - loss: 0.6750 - categorical_accuracy: 0.7109 - val_loss: 1.8953 - val_categorical_accuracy: 0.4100

#### Still Model is Overfitting. Lets add one more layer and increase dropout value

In [21]:
model = Sequential()

model.add(Conv3D(16, (3, 3, 3),padding='same' ,input_shape=(20,120,120,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))

model.add(Conv3D(32, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))

model.add(Conv3D(64, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))

model.add(Conv3D(128, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))



model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(5,activation='softmax'))

In [22]:
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 20, 120, 120, 16)  1312      
_________________________________________________________________
activation_19 (Activation)   (None, 20, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 20, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 10, 60, 60, 16)    0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 10, 60, 60, 16)    0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 10, 60, 60, 32)    13856     
_________________________________________________________________
activation_20 (Activation)   (None, 10, 60, 60, 32)    0         
__________

In [23]:
model_name = 'model_init3' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [24]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
11/11 [==============================] - 59s 5s/step - loss: 2.1032 - categorical_accuracy: 0.3138 - val_loss: 6.9435 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init3_2020-12-2811_48_09.796980/model-00001-2.11251-0.31674-6.94345-0.22000.h5
Epoch 2/10
11/11 [==============================] - 47s 4s/step - loss: 1.6009 - categorical_accuracy: 0.4490 - val_loss: 6.0910 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to model_init3_2020-12-2811_48_09.796980/model-00002-1.60214-0.44495-6.09096-0.18000.h5
Epoch 3/10
11/11 [==============================] - 48s 4s/step - loss: 1.3515 - categorical_accuracy: 0.4772 - val_loss: 6.4275 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init3_2020-12-2811_48_09.796980/model-00003-1.33572-0.47964-6.42754-0.20000.h5
Epoch 4/10
11/11 [==============================] - 46s 4s/step - loss: 1.2165 - categorical_accuracy: 0.5229 - val_loss: 6.6236 - val_categorical_accuracy: 0.2300

#### No Improvement in the model. Lets try different Architecture

### CNN - LSTM

In [25]:
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),input_shape=(20,120,120,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))


model.add(TimeDistributed(Flatten()))


model.add(LSTM(128))
model.add(Dropout(0.5))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))

In [26]:
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 20, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 20, 30, 30, 64)    18496     
__________

In [27]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [28]:
model_name = 'model_init4' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [29]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 64
Source path =  ./Project_data/train ; batch size = 64
Epoch 1/10
11/11 [==============================] - 60s 5s/step - loss: 1.6735 - categorical_accuracy: 0.2802 - val_loss: 1.5284 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init4_2020-12-2811_48_09.796980/model-00001-1.67385-0.27149-1.52838-0.35000.h5
Epoch 2/10
11/11 [==============================] - 42s 4s/step - loss: 1.5462 - categorical_accuracy: 0.3173 - val_loss: 1.4229 - val_categorical_accuracy: 0.4800

Epoch 00002: saving model to model_init4_2020-12-2811_48_09.796980/model-00002-1.55772-0.31071-1.42295-0.48000.h5
Epoch 3/10
11/11 [==============================] - 45s 4s/step - loss: 1.4428 - categorical_accuracy: 0.3534 - val_loss: 1.3189 - val_categorical_accuracy: 0.5300

Epoch 00003: saving model to model_init4_2020-12-2811_48_09.796980/model-00003-1.45073-0.34389-1.31887-0.53000.h5
Epoch 4/10
11/11 [==============================] - 45s 4

#### Now, Model is not overfitted, but got only 66% validation accuracy. Lets increase the no. of layers and epoch

In [30]:
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),input_shape=(20,120,120,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))


model.add(TimeDistributed(Flatten()))


model.add(LSTM(128))
model.add(Dropout(0.5))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))

In [31]:
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_14 (TimeDis (None, 20, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_15 (TimeDis (None, 20, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_16 (TimeDis (None, 20, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, 20, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 20, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_19 (TimeDis (None, 20, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 20, 30, 30, 64)    18496     
__________

In [32]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [33]:
model_name = 'model_init5' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [34]:
num_epochs = 15
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 64
Source path =  ./Project_data/train ; batch size = 64
Epoch 1/15
11/11 [==============================] - 59s 5s/step - loss: 1.6927 - categorical_accuracy: 0.2272 - val_loss: 1.5402 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init5_2020-12-2811_48_09.796980/model-00001-1.68527-0.22775-1.54018-0.29000.h5
Epoch 2/15
11/11 [==============================] - 43s 4s/step - loss: 1.4808 - categorical_accuracy: 0.3487 - val_loss: 1.4166 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init5_2020-12-2811_48_09.796980/model-00002-1.49004-0.34389-1.41665-0.40000.h5
Epoch 3/15
11/11 [==============================] - 45s 4s/step - loss: 1.3809 - categorical_accuracy: 0.4125 - val_loss: 1.3879 - val_categorical_accuracy: 0.4100

Epoch 00003: saving model to model_init5_2020-12-2811_48_09.796980/model-00003-1.38949-0.40875-1.38790-0.41000.h5
Epoch 4/15
11/11 [==============================] - 45s 4

#### Cool, Now we got 68% validation accuracy after increase the layer. But now lets try CNN - GRU

### CNN - GRU

In [35]:
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),input_shape=(20,120,120,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        

model.add(TimeDistributed(Flatten()))


model.add(GRU(128))
model.add(Dropout(0.5))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))
        


from keras.optimizers import Adam
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_30 (TimeDis (None, 20, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_31 (TimeDis (None, 20, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_32 (TimeDis (None, 20, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_33 (TimeDis (None, 20, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_34 (TimeDis (None, 20, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_35 (TimeDis (None, 20, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_36 (TimeDis (None, 20, 30, 30, 64)    18496     
__________

In [36]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [37]:
model_name = 'model_init6' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [38]:
num_epochs = 15
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 64
Source path =  ./Project_data/train ; batch size = 64
Epoch 1/15
11/11 [==============================] - 57s 5s/step - loss: 1.9765 - categorical_accuracy: 0.2406 - val_loss: 1.4721 - val_categorical_accuracy: 0.4000

Epoch 00001: saving model to model_init6_2020-12-2811_48_09.796980/model-00001-1.98279-0.24434-1.47208-0.40000.h5
Epoch 2/15
11/11 [==============================] - 43s 4s/step - loss: 1.5500 - categorical_accuracy: 0.3436 - val_loss: 1.4501 - val_categorical_accuracy: 0.3600

Epoch 00002: saving model to model_init6_2020-12-2811_48_09.796980/model-00002-1.55125-0.34087-1.45009-0.36000.h5
Epoch 3/15
11/11 [==============================] - 46s 4s/step - loss: 1.4768 - categorical_accuracy: 0.3575 - val_loss: 1.3573 - val_categorical_accuracy: 0.3700

Epoch 00003: saving model to model_init6_2020-12-2811_48_09.796980/model-00003-1.48056-0.36048-1.35729-0.37000.h5
Epoch 4/15
11/11 [==============================] - 46s 4

#### Got 63% Validation accuracy. Lets Increase the no. of layers

In [39]:
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),input_shape=(20,120,120,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        

model.add(TimeDistributed(Flatten()))


model.add(GRU(128))
model.add(Dropout(0.5))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))
        


from keras.optimizers import Adam
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_43 (TimeDis (None, 20, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_44 (TimeDis (None, 20, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_45 (TimeDis (None, 20, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_46 (TimeDis (None, 20, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_47 (TimeDis (None, 20, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_48 (TimeDis (None, 20, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_49 (TimeDis (None, 20, 30, 30, 64)    18496     
__________

In [40]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [41]:
model_name = 'model_init7' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [42]:
num_epochs = 15
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 64
Source path =  ./Project_data/trainEpoch 1/15
 ; batch size = 64
11/11 [==============================] - 59s 5s/step - loss: 1.7185 - categorical_accuracy: 0.2998 - val_loss: 1.4345 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init7_2020-12-2811_48_09.796980/model-00001-1.72435-0.29713-1.43450-0.35000.h5
Epoch 2/15
11/11 [==============================] - 45s 4s/step - loss: 1.5104 - categorical_accuracy: 0.3632 - val_loss: 1.4447 - val_categorical_accuracy: 0.3300

Epoch 00002: saving model to model_init7_2020-12-2811_48_09.796980/model-00002-1.49109-0.36652-1.44469-0.33000.h5
Epoch 3/15
11/11 [==============================] - 48s 4s/step - loss: 1.3610 - categorical_accuracy: 0.4178 - val_loss: 1.3384 - val_categorical_accuracy: 0.4700

Epoch 00003: saving model to model_init7_2020-12-2811_48_09.796980/model-00003-1.35423-0.41931-1.33838-0.47000.h5
Epoch 4/15
11/11 [==============================] - 48s 4

#### Still Got only 68% validation accuracy. Lets go with again simple models with more layers

In [55]:
model = Sequential()


model.add(Conv3D(16, (3, 3, 3),padding='same' ,input_shape=(20,120,120,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))

model.add(Conv3D(16, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))

model.add(Conv3D(32, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))

model.add(Conv3D(32, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))



model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(5,activation='softmax'))

In [56]:
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_50 (Conv3D)           (None, 20, 120, 120, 16)  1312      
_________________________________________________________________
activation_50 (Activation)   (None, 20, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_74 (Batc (None, 20, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_50 (MaxPooling (None, 10, 60, 60, 16)    0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 10, 60, 60, 16)    0         
_________________________________________________________________
conv3d_51 (Conv3D)           (None, 10, 60, 60, 16)    6928      
_________________________________________________________________
activation_51 (Activation)   (None, 10, 60, 60, 16)    0         
__________

In [57]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [58]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Source path =  ./Project_data/train ; batch size = 64
Epoch 1/10
./Project_data/val ; batch size = 64
11/11 [==============================] - 60s 5s/step - loss: 2.3694 - categorical_accuracy: 0.2404 - val_loss: 2.3929 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init7_2020-12-2811_48_09.796980/model-00001-2.38910-0.23680-2.39289-0.21000.h5
Epoch 2/10
11/11 [==============================] - 42s 4s/step - loss: 1.8942 - categorical_accuracy: 0.3341 - val_loss: 3.4615 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init7_2020-12-2811_48_09.796980/model-00002-1.89179-0.33333-3.46154-0.22000.h5
Epoch 3/10
11/11 [==============================] - 47s 4s/step - loss: 1.6609 - categorical_accuracy: 0.4150 - val_loss: 4.2769 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init7_2020-12-2811_48_09.796980/model-00003-1.67887-0.41629-4.27690-0.25000.h5
Epoch 4/10
11/11 [==============================] - 46s 4

#### Ahhh!! We got very Less accuracy. Lets remove dropout

In [59]:
model = Sequential()


model.add(Conv3D(16, (3, 3, 3),padding='same' ,input_shape=(20,120,120,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


model.add(Conv3D(16, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


model.add(Conv3D(32, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (3, 3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(5,activation='softmax'))

In [60]:
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_54 (Conv3D)           (None, 20, 120, 120, 16)  1312      
_________________________________________________________________
activation_54 (Activation)   (None, 20, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_79 (Batc (None, 20, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_54 (MaxPooling (None, 10, 60, 60, 16)    0         
_________________________________________________________________
conv3d_55 (Conv3D)           (None, 10, 60, 60, 16)    6928      
_________________________________________________________________
activation_55 (Activation)   (None, 10, 60, 60, 16)    0         
_________________________________________________________________
batch_normalization_80 (Batc (None, 10, 60, 60, 16)    64        
__________

In [61]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [62]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 64
Source path =  ./Project_data/train ; batch size = 64
Epoch 1/10
11/11 [==============================] - 60s 5s/step - loss: 1.5904 - categorical_accuracy: 0.4028 - val_loss: 1.6609 - val_categorical_accuracy: 0.5300

Epoch 00001: saving model to model_init7_2020-12-2811_48_09.796980/model-00001-1.61326-0.39367-1.66089-0.53000.h5
Epoch 2/10
11/11 [==============================] - 43s 4s/step - loss: 0.8622 - categorical_accuracy: 0.6864 - val_loss: 1.5877 - val_categorical_accuracy: 0.5100

Epoch 00002: saving model to model_init7_2020-12-2811_48_09.796980/model-00002-0.85882-0.69080-1.58766-0.51000.h5
Epoch 3/10
11/11 [==============================] - 48s 4s/step - loss: 0.5700 - categorical_accuracy: 0.7903 - val_loss: 1.2292 - val_categorical_accuracy: 0.5400

Epoch 00003: saving model to model_init7_2020-12-2811_48_09.796980/model-00003-0.57726-0.78582-1.22918-0.54000.h5
Epoch 4/10
11/11 [==============================] - 47s 4

#### Without dropout model is overfit again

### CNN + GRU

In [9]:
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape=(20,120,120,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        

model.add(TimeDistributed(Flatten()))


model.add(GRU(128))
model.add(Dropout(0.5))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
        
model.add(Dense(5, activation='softmax'))
        


from keras.optimizers import Adam
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 20, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 20, 30, 30, 64)    18496     
__________

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [14]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 75
Source path =  ./Project_data/train ; batch size = 75
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


9/9 [==============================] - 68s 8s/step - loss: 2.0004 - categorical_accuracy: 0.2150 - val_loss: 1.4828 - val_categorical_accuracy: 0.4500

Epoch 00001: saving model to model_init_2020-12-2715_09_40.789923/model-00001-2.00078-0.21569-1.48275-0.45000.h5
Epoch 2/20
9/9 [==============================] - 47s 5s/step - loss: 1.6252 - categorical_accuracy: 0.3026 - val_loss: 1.4045 - val_categorical_accuracy: 0.4800

Epoch 00002: saving model to model_init_2020-12-2715_09_40.789923/model-00002-1.62636-0.30317-1.40452-0.48000.h5
Epoch 3/20
9/9 [==============================] - 52s 6s/step - loss: 1.4515 - categorical_accuracy: 0.3752 - val_loss: 1.4040 - val_categorical_accuracy: 0.3500

Epoch 00003: saving model to model_init_2020-12-2715_09_40.789923/model-00003-1.45511-0.37406-1.40400-0.35000.h5
Epoch 4/20
9/9 [==============================] - 51s 6s/step - loss: 1.3099 - categorical_accuracy: 0.4459 - val_loss: 1.3662 - val_categorical_accuracy: 0.4000

Epoch 00004: saving 

#### We got 78% validation accuracy and 83% train accuracy at the 19 epoch

### Recommendation for increase the accuracy
As we try different differnet __Architectures__ of the __Model__ but still got __78% validation accuracy__. 
We can __increase accuracy__ by increase the __image resolution__ and __no. of frames__. Here we are using __120x120 image size and 20 frames__. We can try __140x140 or 160X160 image size and use 25 or 30 frames__ to __increase__ the __accuracy__.